In [3]:
import torch
import os
from PIL import Image
from transformers import AutoTokenizer, AutoModelForCausalLM
from urllib.request import urlopen
import torch.nn as nn
from huggingface_hub import hf_hub_download

from models import CLIPVisionTower

DEVICE = "cuda:0"
PROMPT = "This is a dialog with AI assistant.\n"

# Change paths to yours
ckkpt_path = "../ckpts/orig-v19_2000/model"
test_json_path = "../data/OMNIGROUND_CWB_flickr30k_train.json"

tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", use_fast=False)
model = AutoModelForCausalLM.from_pretrained(os.path.join(ckkpt_path, "tuned-model"), torch_dtype=torch.bfloat16, device_map=DEVICE)
projection = torch.load(os.path.join(ckkpt_path, "projection.pt"), map_location=DEVICE)
special_embs = torch.load(os.path.join(ckkpt_path,"special_embeddings.pt"), map_location=DEVICE)

clip = CLIPVisionTower("openai/clip-vit-large-patch14-336")
clip.load_model()
clip = clip.to(device=DEVICE, dtype=torch.bfloat16)

# there is a difference with running without eval mode
model.eval()
clip.eval()
print("Models loaded")

Models loaded


In [4]:
import re

@torch.inference_mode()
def gen_answer(model, tokenizer, clip, projection, query, special_embs, image=None):
    bad_words_ids = tokenizer(["\n", "</s>", ":"], add_special_tokens=False).input_ids + [[13]]
    gen_params = {
        "do_sample": False,
        "max_new_tokens": 150,
        "early_stopping": True,
        "num_beams": 3,
        "repetition_penalty": 1.0,
        "remove_invalid_values": True,
        "eos_token_id": 2,
        "pad_token_id": 2,
        "forced_eos_token_id": 2,
        "use_cache": True,
        "no_repeat_ngram_size": 4,
        "bad_words_ids": bad_words_ids,
        "num_return_sequences": 1,
    }
    with torch.no_grad():
        image_features = clip.image_processor(image.resize((336, 336)), return_tensors='pt', do_center_crop=False)
        image_embedding = clip(image_features['pixel_values']).to(device=DEVICE, dtype=torch.bfloat16)

        projected_vision_embeddings = projection(image_embedding).to(device=DEVICE, dtype=torch.bfloat16)
        prompt_ids = tokenizer.encode(f"{PROMPT}", add_special_tokens=False, return_tensors="pt").to(device=DEVICE)
        question_ids = tokenizer.encode(query, add_special_tokens=False, return_tensors="pt").to(device=DEVICE)

        prompt_embeddings = model.model.embed_tokens(prompt_ids).to(torch.bfloat16)
        question_embeddings = model.model.embed_tokens(question_ids).to(torch.bfloat16)

        embeddings = torch.cat(
            [
                prompt_embeddings,
                special_embs['SOI'][None, None, ...],
                projected_vision_embeddings,
                special_embs['EOI'][None, None, ...],
                special_embs['USER'][None, None, ...],
                question_embeddings,
                special_embs['BOT'][None, None, ...]
            ],
            dim=1,
        ).to(dtype=torch.bfloat16, device=DEVICE)
        out = model.generate(inputs_embeds=embeddings, **gen_params)
    out = out[:, :-1]
    generated_texts = tokenizer.batch_decode(out)[0]
    return generated_texts


def process_sample(sample):
    question = sample["conversations"][0]["value"]
    gt_ans = sample["conversations"][1]["value"]
    img_path = os.path.join("../data", os.path.join(sample["image"]))
    img = Image.open(img_path)

    answer = gen_answer(
        model,
        tokenizer,
        clip,
        projection,
        query=question,
        special_embs=special_embs,
        image=img
    )

    return question, answer, gt_ans, img

def correct_bbox(bbox):
    left, top, right, bottom = bbox
    
    new_bottom = min(max(top, bottom), 1)
    new_top = min(top, bottom)
    
    new_left = min(left, right)
    new_right = min(max(left, right), 1)
    
    return new_left, new_top, new_right, new_bottom

def get_iou(bbox1, bbox2):
    x1, y1, x2, y2 = bbox1
    x3, y3, x4, y4 = bbox2
    x = max(x1, x3)
    y = max(y1, y3)
    w = min(x2, x4) - x
    h = min(y2, y4) - y
    if w < 0 or h < 0:
        return 0
    return w * h / ((x2 - x1) * (y2 - y1) + (x4 - x3) * (y4 - y3) - w * h)

def rel_to_abs(bbox, width, height):
    left = width * bbox[0]
    top = height * bbox[1]
    right = width * bbox[2]
    bottom = height * bbox[3]
    
    return left, top, right, bottom

def get_bbox(ans):
    matches = re.search(r'\[(.*?)\]', ans)
    if matches:
        bbox = list(map(float, matches.group(1).split(',')))
        #print(bbox)
        bbox = correct_bbox(bbox)
        return bbox
    else:
        return [0, 0, 0, 0]

In [5]:
import json
from PIL import Image, ImageDraw
import numpy as np
from tqdm.notebook import tqdm

test_data = json.loads(open(test_json_path, "r").read())
acc05 = 0
N = 1
A = 100
found = 0

attempt = 0
bar = tqdm(range(A, A+N), desc=f"Acc@05 = {0}:")
for sample_id in bar:
    for attempt in range(1):
        try:
            sample = test_data[sample_id]
            #sample["image"] = os.path.join("coco", os.path.join(sample["image"]))
            
            question, answer, gt_ans, img = process_sample(sample)

            # question += " Provide bounding box coordinates in format: [left, top, right, bottom]."
            
            width, height = img.size
            
            bbox_gt = get_bbox(gt_ans)
            bbox_gt_coorts = rel_to_abs(bbox_gt, width, height)
            
            bbox_ans = get_bbox(answer)
            bbox_ans_coors = rel_to_abs(bbox_ans, width, height)
            
            iou = get_iou(bbox_ans, bbox_gt)
            found += 1
            
            if iou>0:
                break
        except ValueError as ex:
            print(f"Incorrect output format. Sample id: {sample_id}; answer: {answer}; error: {ex} ")
            bbox_ans = [0, 0, 0, 0]
            iou = 0
    
    if iou>0.5:
        acc05+=1
    found += 1
    bar.set_description(f"Acc@05 = {np.round(acc05/found, 3)}")

    draw = ImageDraw.Draw(img)
    draw.rectangle(bbox_ans_coors, outline="red", width=2)
    draw.rectangle(bbox_gt_coorts, outline="green", width=2)

    print("Sample id:", sample_id)
    print("Attempts:", attempt)
    print("Question:", question)
    print("Answer:", answer)
    print("Gt answer:", gt_ans)
    print("\n")
    print("IoU metrics:", iou)
    
    img.show()  # Отображение
    
      
print("Accuracy@0.5:", acc05/found)
print("Accuracy@0.5:", acc05/N) 
# interesting samples
# [101, 103, 105, 120, 123, 124, 200, 202, 211, 501, 601, 607, 606]

Acc@05 = 0::   0%|          | 0/1 [00:00<?, ?it/s]

Sample id: 100
Attempts: 0
Question: Provide the bounding box for a young skateboarder jumping a trick in the air.
Answer: [0.3, 0.2, 1.0, 2.0].
Gt answer: [0.276, 0.388, 0.712, 0.672].


IoU metrics: 0.20643030542539378


Accuracy@0.5: 0.0
Accuracy@0.5: 0.0


In [6]:
sample

{'id': 1716,
 'image': 'flickr30k/3456579559.jpg',
 'conversations': [{'from': 'human',
   'value': 'Provide the bounding box for a young skateboarder jumping a trick in the air.'},
  {'from': 'gpt', 'value': '[0.276, 0.388, 0.712, 0.672].'}]}

In [ ]:
# что вылезает: 
# [0.5, 0.6, 0.<|current_max|>, 0.9].
# [0.0, 0.5, 0.<|assistant|>, [0.4, 0., 1.0].
# мб проблема с llm

# Отличие: ресайз изображения до (336, 336) + do_center_crop=False при подаче в image_processor!